In [1]:
import pandas as pd 
from bs4 import BeautifulSoup 
import requests
import json
import re
import urllib
from time import sleep

In [2]:
# Team URL Extraction
def build_team_urls():
    # Open the espn teams webpage and extract the names of each roster available.
    f = urllib.request.urlopen('http://www.espn.com/nba/teams')
    teams_source = f.read().decode('utf-8')
    teams = dict(re.findall("www\.espn\.com/nba/team/_/name/(\w+)/(.+?)\",", teams_source))
    # Using the names of the rosters, create the urls of each roster
    roster_urls = []
    for key in teams.keys():
        # each roster webpage follows this general pattern.
        roster_urls.append('http://www.espn.com/nba/team/roster/_/name/' + key + '/' + teams[key])
        teams[key] = str(teams[key])
    return dict(zip(teams.values(), roster_urls))
team_urls = build_team_urls()

In [3]:
#Extract Players basic information
def player_info(team_url):
    f = urllib.request.urlopen(team_url)
    teams_source = f.read().decode('utf-8')
    sleep(0.5)
    player_regex = ('"athletes":(.+?)"subType":')
    player = re.findall(player_regex, teams_source)[0]
    # Convert palyers to pandas dataframe
    return pd.read_json(player[1:-4], lines=True)

In [4]:
team_urls.values()

dict_values(['http://www.espn.com/nba/team/roster/_/name/bos/boston-celtics', 'http://www.espn.com/nba/team/roster/_/name/bkn/brooklyn-nets', 'http://www.espn.com/nba/team/roster/_/name/ny/new-york-knicks', 'http://www.espn.com/nba/team/roster/_/name/phi/philadelphia-76ers', 'http://www.espn.com/nba/team/roster/_/name/tor/toronto-raptors', 'http://www.espn.com/nba/team/roster/_/name/chi/chicago-bulls', 'http://www.espn.com/nba/team/roster/_/name/cle/cleveland-cavaliers', 'http://www.espn.com/nba/team/roster/_/name/det/detroit-pistons', 'http://www.espn.com/nba/team/roster/_/name/ind/indiana-pacers', 'http://www.espn.com/nba/team/roster/_/name/mil/milwaukee-bucks', 'http://www.espn.com/nba/team/roster/_/name/atl/atlanta-hawks', 'http://www.espn.com/nba/team/roster/_/name/cha/charlotte-hornets', 'http://www.espn.com/nba/team/roster/_/name/mia/miami-heat', 'http://www.espn.com/nba/team/roster/_/name/orl/orlando-magic', 'http://www.espn.com/nba/team/roster/_/name/wsh/washington-wizards', '

In [5]:
all_players_df= []
for url in team_urls.values():
    all_players_df.append(player_info(url))
all_players_df = pd.concat(all_players_df)

In [18]:
all_players_df = all_players_df.reset_index().drop(columns=['index'])

In [21]:
career_stats_df = pd.DataFrame(columns = ["GP","GS","MIN","FGM", "FGA","FG%","3PTM",
                                          "3PTA","3P%","FTM","FTA","FT%","OR","DR",
                                          "REB","AST","BLK","STL","PF","TO","PTS"])
# Performance score
for player_index in all_players_df.index:
    url = "https://www.espn.com/nba/player/stats/_/id/" + str(all_players_df.iloc[player_index]['id'])
    f = urllib.request.urlopen(url)
    sleep(0.3)
    player_source = f.read().decode('utf-8')
    # extract career stats using this regex
    stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
    career_info = re.findall(stats_regex, player_source)
    try:
        # convert the stats to a list of floats, and add the entry to the DataFrame
        career_info = career_info[0].replace("\"", "").split(",")
        career_info = list(chain.from_iterable([i.split("-") for i in career_info]))
        career_info = list(map(float,career_info))
        career_stats_df = career_stats_df.append(pd.Series(career_info, index = career_stats_df.columns, name=player_index))
    except:
        # if no career stats were returned, the player was a rookie with no games played
        print(str(player_index) + " has no info, ", end = "")

54 has no info, 120 has no info, 467 has no info, 

In [24]:
career_stats_df

,GP,GS,MIN,FGM,FGA,FG%,3PTM,3PTA,3P%,FTM,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,279.0,172.0,26.3,4.9,10.4,46.8,1.4,3.8,37.1,1.9,...,67.8,0.9,3.6,4.5,1.4,0.3,0.9,2.4,1.5,13.1
1,37.0,0.0,9.5,1.1,3.5,32.8,0.6,2.1,31.6,0.4,...,68.4,0.2,1.1,1.3,0.6,0.1,0.3,0.9,0.4,3.3
2,7.0,0.0,4.7,1.6,2.0,78.6,0.0,0.0,0.0,0.1,...,33.3,0.1,2.0,2.1,0.1,0.6,0.1,1.0,0.4,3.3
3,48.0,2.0,9.8,1.3,2.6,50.0,0.2,0.7,27.3,0.6,...,66.7,0.5,1.4,1.9,0.5,0.2,0.5,0.9,0.5,3.4
4,641.0,479.0,30.8,5.3,11.6,45.1,1.3,3.6,36.6,3.5,...,82.3,0.8,3.6,4.4,3.5,0.4,1.0,1.8,2.0,15.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,3.0,1.0,16.0,1.7,5.3,31.3,1.0,2.7,37.5,1.0,...,75.0,0.3,3.0,3.3,2.0,0.7,0.0,2.0,0.7,5.3
502,78.0,12.0,14.2,2.2,5.2,41.7,0.6,1.5,40.4,0.7,...,73.2,0.3,1.7,2.0,1.0,0.2,0.5,1.1,0.6,5.6
503,11.0,0.0,7.1,0.5,1.5,29.4,0.1,0.5,20.0,0.1,...,50.0,0.1,0.5,0.6,1.0,0.1,0.3,0.7,0.5,1.1
504,152.0,75.0,23.3,3.5,7.4,46.9,0.9,2.4,36.4,1.9,...,81.6,0.5,2.8,3.3,3.4,0.7,0.8,2.0,1.3,9.8


In [25]:
# join and clean the data
all_stats_df = all_players_df.join(career_stats_df)
def convert_height(height):
    split_height = height.split(" ")
    feet = float(split_height[0].replace("\'",""))
    inches = float(split_height[1].replace("\"",""))
    return (feet*12 + inches)
all_stats_df['height'] = [convert_height(x) for x in all_stats_df['height']]
all_stats_df['weight'] = [float(x.split(" ")[0]) for x in all_stats_df['weight']]
all_stats_df['salary'] = [int(re.sub(r'[^\d.]+', '', s)) if isinstance(s, str) else s for s in all_stats_df['salary'].values]



In [26]:
all_stats_df

,name,href,uid,guid,id,height,weight,age,position,jersey,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,Jaylen Brown,https://www.espn.com/nba/player/_/id/3917376/j...,s:40~l:46~a:3917376,0d5cde01f6d3225fdae544ef3304cda2,3917376,78.0,223.0,23,SG,7.0,...,67.8,0.9,3.6,4.5,1.4,0.3,0.9,2.4,1.5,13.1
1,Carsen Edwards,https://www.espn.com/nba/player/_/id/4066407/c...,s:40~l:46~a:4066407,4b8ebdfd01221567925035c1e0d0c337,4066407,71.0,200.0,22,PG,4.0,...,68.4,0.2,1.1,1.3,0.6,0.1,0.3,0.9,0.4,3.3
2,Tacko Fall,https://www.espn.com/nba/player/_/id/3904625/t...,s:40~l:46~a:3904625,cec7bd80fe2880fe8d9efe9a2294a563,3904625,89.0,311.0,24,C,99.0,...,33.3,0.1,2.0,2.1,0.1,0.6,0.1,1.0,0.4,3.3
3,Javonte Green,https://www.espn.com/nba/player/_/id/2596112/j...,s:40~l:46~a:2596112,a4940ed033e0a114e8862f5a094aa3f8,2596112,76.0,205.0,27,SG,43.0,...,66.7,0.5,1.4,1.9,0.5,0.2,0.5,0.9,0.5,3.4
4,Gordon Hayward,https://www.espn.com/nba/player/_/id/4249/gord...,s:40~l:46~a:4249,56f675cb8f40a5aaee5f5747ec9099c5,4249,79.0,225.0,30,SF,20.0,...,82.3,0.8,3.6,4.4,3.5,0.4,1.0,1.8,2.0,15.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Luka Samanic,https://www.espn.com/nba/player/_/id/4423309/l...,s:40~l:46~a:4423309,bc9995ea4b1a9782ef30b0a4d7299aad,4423309,82.0,227.0,20,PF,19.0,...,75.0,0.3,3.0,3.3,2.0,0.7,0.0,2.0,0.7,5.3
502,Lonnie Walker IV,https://www.espn.com/nba/player/_/id/4277890/l...,s:40~l:46~a:4277890,b728fc4662fe032160f7659e2e05f4c4,4277890,77.0,204.0,21,SG,1.0,...,73.2,0.3,1.7,2.0,1.0,0.2,0.5,1.1,0.6,5.6
503,Quinndary Weatherspoon,https://www.espn.com/nba/player/_/id/3907358/q...,s:40~l:46~a:3907358,f40458f631cee819bf79e83079dc3aac,3907358,75.0,205.0,24,SG,15.0,...,50.0,0.1,0.5,0.6,1.0,0.1,0.3,0.7,0.5,1.1
504,Derrick White,https://www.espn.com/nba/player/_/id/3078576/d...,s:40~l:46~a:3078576,8aba42b74c1e6ccd315097691c299668,3078576,76.0,190.0,26,PG,4.0,...,81.6,0.5,2.8,3.3,3.4,0.7,0.8,2.0,1.3,9.8
